# Breadboard for prototyping FEMA NRI data

In [1]:
# import libraries
# NOTE: global_vars should be edited to include local paths and credentials before use. 
# If global_vars.py is created in the root dir remove the ignore/ prefix in the import statement below.
import ignore.global_vars as gv
from sqlalchemy import create_engine
import pandas as pd
import db_tools as dbt
import zipfile
import os
import geopandas as gpd
from IPython.display import HTML, display
from scipy.interpolate import griddata

In [2]:
# NOTE: Set to True if a full rebuild is required, set to False to skip table builds. Search 'REBUILD_TABLE' to see which sections are effected.
REBUILD = True

In [3]:
if REBUILD:
    # Database connection setup
    dbt.connection(test=True)
    dbt.engine()
else:
    print('Rebuild skipped')

Connection established and verified with test
Created SQLAlchemy engine for disaster_db


In [4]:
if REBUILD:
    # List zip file contents  
    zip_path = gv.DATA_PATHS["nri_shapefile"]
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        files = [f for f in zip_ref.namelist() if not f.endswith('/')]
        print("Key files:", [f for f in files if f.endswith(('.shp', '.dbf', '.csv'))])
else:
    print("Rebuild skipped")

Key files: ['NRI_HazardInfo.csv', 'NRI_Shapefile_CensusTracts.dbf', 'NRI_Shapefile_CensusTracts.shp', 'NRIDataDictionary.csv']


In [5]:
# Unzip the shapefile components
if REBUILD:
    zip_path = gv.DATA_PATHS["nri_shapefile"]
    extract_dir = gv.DATA_PATHS["extract_dir"]

    os.makedirs(extract_dir, exist_ok=True)
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_dir)

    # Extract .shp file path
    shp_path = os.path.join(extract_dir, "NRI_Shapefile_CensusTracts.shp")

    # Load shapefile
    gdf = gpd.read_file(shp_path)
else:
    print("Rebuild skipped")


In [6]:
if REBUILD:
    gdf.to_postgis(name="nri_shape_census_tracts", con=dbt.engine(), if_exists="replace")
else:
    print("Rebuild skipped")

Created SQLAlchemy engine for disaster_db


In [7]:
# DB check
df= pd.read_sql('SELECT * FROM nri_shape_census_tracts WHERE "STATEABBRV" = \'AZ\' LIMIT 5;', dbt.engine())
df.head()

Created SQLAlchemy engine for disaster_db


,NRI_ID,STATE,STATEABBRV,STATEFIPS,COUNTY,COUNTYTYPE,COUNTYFIPS,STCOFIPS,TRACT,TRACTFIPS,...,WNTW_ALRP,WNTW_ALRA,WNTW_ALR_N,WNTW_RISKV,WNTW_RISKS,WNTW_RISKR,NRI_VER,Shape_Leng,Shape_Area,geometry
0,T04001942600,Arizona,AZ,04,Apache,County,001,04001,942600,04001942600,...,3.804119e-08,0.000000e+00,50.871362,1213.560266,45.950863,Relatively Low,March 2023,290407.819899,2.380489e+09,0103000020110F000001000000DF0C0000E9B7AF6F4D4B...
1,T04001942700,Arizona,AZ,04,Apache,County,001,04001,942700,04001942700,...,4.457586e-08,2.923576e-12,56.263945,4581.281047,72.481330,Relatively Moderate,March 2023,616060.118692,4.670224e+09,0103000020110F0000010000001D240000D8F0F4EC2C27...
2,T04001944000,Arizona,AZ,04,Apache,County,001,04001,944000,04001944000,...,7.223335e-08,0.000000e+00,66.256430,8569.800761,83.361081,Relatively Moderate,March 2023,233719.792980,1.211321e+09,0103000020110F000001000000A1100000C0EC9EF23927...
3,T04001944100,Arizona,AZ,04,Apache,County,001,04001,944100,04001944100,...,4.621456e-08,2.797498e-12,57.054278,5823.210780,76.915759,Relatively Moderate,March 2023,492964.995655,2.836179e+09,0103000020110F00000100000079240000AAF1D2BB3627...
4,T04001944201,Arizona,AZ,04,Apache,County,001,04001,944201,04001944201,...,3.807192e-08,0.000000e+00,50.924208,3514.540318,67.258479,Relatively Moderate,March 2023,144035.896648,7.723450e+08,0103000020110F0000010000005F080000FE65F7E4CC45...


In [8]:
# Query to get column names
df_cols=pd.read_sql(
    '''
    SELECT column_name
    FROM information_schema.columns
    WHERE table_schema='public' AND table_name='nri_shape_census_tracts'
    ORDER BY ordinal_position
    ''', dbt.engine()
)

html = df_cols.to_html(index=False)
display(
    HTML(
        f'<div style="max-height:400px; overflow:auto; border:1px solid #ddd">{html}</div>'
    )
)

Created SQLAlchemy engine for disaster_db


column_name
NRI_ID
STATE
STATEABBRV
STATEFIPS
COUNTY
COUNTYTYPE
COUNTYFIPS
STCOFIPS
TRACT
TRACTFIPS


In [9]:
# US visualization for tract view
df_risk = gpd.read_postgis(
    'SELECT "RISK_VALUE","TRACTFIPS", geometry FROM nri_shape_census_tracts;',
    dbt.engine(),
    geom_col='geometry'
)
df_risk.head()

Created SQLAlchemy engine for disaster_db


,RISK_VALUE,TRACTFIPS,geometry
0,3.316283e+06,04003000201,"POLYGON ((-12208530.554 3819453.965, -12208530..."
1,1.594669e+06,04003000202,"POLYGON ((-12218992.251 3804501.365, -12219082..."
2,4.066984e+06,04003000203,"POLYGON ((-12218005.958 3788974.981, -12217845..."
3,8.537266e+05,04003000301,"POLYGON ((-12275082.153 3758768.382, -12275081..."
4,5.970729e+05,04003000303,"POLYGON ((-12259470.262 3819462.268, -12259469..."


In [10]:
df_risk.shape

(85154, 3)